In [ ]:
# Imports
import urllib.request  
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import io
import math
import pandas as pd
import numpy as np
import glob

In [ ]:
#Natural Language Processing Packages
import re
import nltk

from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
path = r'/content' # use your path (local directory with CSV files in them if you are not using Google Colab)
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
draft_analyst_df = frame.reindex(columns = frame.columns.tolist() + ['CHARACTERVADER', 'SKATINGVADER', 'PUCKSKILLVADER', 'SHOOTINGVADER'])

In [ ]:
draft_analyst_df.head()

,Unnamed: 0,RANK,NAME,SOURCE,CHARACTER,SKATING,PUCK SKILL,SHOOTING,CHARACTERVADER,SKATINGVADER,PUCKSKILLVADER,SHOOTINGVADER
0,0,1,"Lafreniere, Alexis",DraftAnalyst 2020.pdf,['Lafreniere also is a team-first leader who h...,['Lafreniere is an excellent skater in all dir...,"['What makes Lafreniere unique, however, is hi...",['From his deadly wrist shot to his superior p...,NaN,NaN,NaN,NaN
1,1,2,"Byfield, Quinton",DraftAnalyst 2020.pdf,['Byfield combines obvious physical strengths ...,['He is as much a danger in tight spaces as he...,"['On the power play, Byfield threads the needl...","['Byfield, however, is exceptionally crafty wi...",NaN,NaN,NaN,NaN
2,2,3,"Stutzle, Tim",DraftAnalyst 2020.pdf,['His quickness on or off the puck is evident ...,['His first step is explosive; his skating sty...,['His quickness on or off the puck is evident ...,['His quickness on or off the puck is evident ...,NaN,NaN,NaN,NaN
3,3,4,"Perfetti, Cole",DraftAnalyst 2020.pdf,['A lethal scorer who possesses an arsenal of ...,['He is a strong skater with excellent balance...,"['Off the puck, he has very quick hands and a ...",['A lethal scorer who possesses an arsenal of ...,NaN,NaN,NaN,NaN
4,4,5,"Rossi, Marco",DraftAnalyst 2020.pdf,"['Rossi is a consistent breakaway threat, but ...","['Rossi is a consistent breakaway threat, but ...",['A dynamic playmaker with elite finishing ski...,['One of the more exciting forward prospects i...,NaN,NaN,NaN,NaN


In [ ]:
# Initialize the Sentiment Analysis
sid = SentimentIntensityAnalyzer()

In [ ]:
#Generate the Scores
character = draft_analyst_df['CHARACTER'].values
charatcer_score_list = []
for i in character:
  charatcer_score_list.append(sid.polarity_scores(i)['compound'])

skating = draft_analyst_df['SKATING'].values
skating_score_list = []
for i in skating:
  skating_score_list.append(sid.polarity_scores(i)['compound'])

puck_skill = draft_analyst_df['PUCK SKILL'].values
puck_skill_score_list = []
for i in puck_skill:
  puck_skill_score_list.append(sid.polarity_scores(i)['compound'])

shooting_skill = draft_analyst_df['SHOOTING'].values
shooting_skill_score_list = []
for i in shooting_skill:
  shooting_skill_score_list.append(sid.polarity_scores(i)['compound'])

In [ ]:
draft_analyst_df['CHARACTERVADER'] = charatcer_score_list
draft_analyst_df['SKATINGVADER'] = skating_score_list
draft_analyst_df['PUCKSKILLVADER'] = puck_skill_score_list
draft_analyst_df['SHOOTINGVADER'] = shooting_skill_score_list

In [ ]:
draft_analyst_df.head()

,Unnamed: 0,RANK,NAME,SOURCE,CHARACTER,SKATING,PUCK SKILL,SHOOTING,CHARACTERVADER,SKATINGVADER,PUCKSKILLVADER,SHOOTINGVADER
0,0,1,"Lafreniere, Alexis",DraftAnalyst 2020.pdf,['Lafreniere also is a team-first leader who h...,['Lafreniere is an excellent skater in all dir...,"['What makes Lafreniere unique, however, is hi...",['From his deadly wrist shot to his superior p...,0.9640,0.9050,0.8873,0.3818
1,1,2,"Byfield, Quinton",DraftAnalyst 2020.pdf,['Byfield combines obvious physical strengths ...,['He is as much a danger in tight spaces as he...,"['On the power play, Byfield threads the needl...","['Byfield, however, is exceptionally crafty wi...",0.9841,0.7506,0.9878,0.2382
2,2,3,"Stutzle, Tim",DraftAnalyst 2020.pdf,['His quickness on or off the puck is evident ...,['His first step is explosive; his skating sty...,['His quickness on or off the puck is evident ...,['His quickness on or off the puck is evident ...,0.9811,0.7681,0.9597,0.9678
3,3,4,"Perfetti, Cole",DraftAnalyst 2020.pdf,['A lethal scorer who possesses an arsenal of ...,['He is a strong skater with excellent balance...,"['Off the puck, he has very quick hands and a ...",['A lethal scorer who possesses an arsenal of ...,-0.0262,0.9578,0.4902,0.5423
4,4,5,"Rossi, Marco",DraftAnalyst 2020.pdf,"['Rossi is a consistent breakaway threat, but ...","['Rossi is a consistent breakaway threat, but ...",['A dynamic playmaker with elite finishing ski...,['One of the more exciting forward prospects i...,0.9808,0.6310,0.7814,0.8432


Player Type Adjustment

In [ ]:
forward_skill_df = pd.read_excel('Skill Adjustment.xlsx', sheet_name='Forward Adjustments', index_col=0)
defense_skill_df = pd.read_excel('Skill Adjustment.xlsx', sheet_name='Defenseman Adjustments', index_col=0)

In [ ]:
valid = False
while valid is False:
  position = input("Please enter 'F' for forward or 'D' for defense: ")
  if position == 'F':
    valid = True
  elif position == 'D':
    valid = True
  else:
    print("Invalid Input; valid inputs are only 'F' and 'D'")

valid = False
while valid is False:
  if position == 'F':
    player_type = input("Please enter one of the following forward player types - 'TWF' for two-way forward, 'GRN' for energy player, 'PWF' for power forward, 'SNP' for sniper, 'PLY' for playmaker: ")
    if player_type == 'TWF':
      valid = True
    elif player_type == 'GRN':
      valid = True
    elif player_type == 'PWF':
      valid = True
    elif player_type == 'SNP':
      valid = True
    elif player_type == 'PLY':
      valid = True
    else:
      print("Invalid Input; valid inputs are 'TWF', 'GRN', 'PWF', 'SNP', 'PLY'")
  elif position == 'D':
    player_type = input("Please enter one of the following defense player types - 'TWD' for two-way defenseman, 'DFD' for defensive defenseman, 'OFD' for offensive defenseman: ")
    if player_type == 'TWD':
      valid = True
    elif player_type == 'DFD':
      valid = True
    elif player_type == 'OFD':
      valid = True
    else:
      print("Invalid Input; valid inputs are 'TWD', 'OFD', 'DFD'")

draft_analyst_df['CHARACTERVADER'] = charatcer_score_list
draft_analyst_df['SKATINGVADER'] = skating_score_list
draft_analyst_df['PUCKSKILLVADER'] = puck_skill_score_list
draft_analyst_df['SHOOTINGVADER'] = shooting_skill_score_list

if position == 'F':
  if player_type == 'TWF':
    draft_analyst_df['CHARACTERVADER'] *= forward_skill_df.loc["Two-Way","Character"]
    draft_analyst_df['SKATINGVADER'] *= forward_skill_df.loc["Two-Way","Skating"]
    draft_analyst_df['PUCKSKILLVADER'] *= forward_skill_df.loc["Two-Way","Puck Skill"]
    draft_analyst_df['SHOOTINGVADER'] *= forward_skill_df.loc["Two-Way","Shot"]
  elif player_type == 'SNP':
    draft_analyst_df['CHARACTERVADER'] *= forward_skill_df.loc["Sniper","Character"]
    draft_analyst_df['SKATINGVADER'] *= forward_skill_df.loc["Sniper","Skating"]
    draft_analyst_df['PUCKSKILLVADER'] *= forward_skill_df.loc["Sniper","Puck Skill"]
    draft_analyst_df['SHOOTINGVADER'] *= forward_skill_df.loc["Sniper","Shot"]
  elif player_type == 'GRN':
    draft_analyst_df['CHARACTERVADER'] *= forward_skill_df.loc["Energy","Character"]
    draft_analyst_df['SKATINGVADER'] *= forward_skill_df.loc["Energy","Skating"]
    draft_analyst_df['PUCKSKILLVADER'] *= forward_skill_df.loc["Energy","Puck Skill"]
    draft_analyst_df['SHOOTINGVADER'] *= forward_skill_df.loc["Energy","Shot"]
  elif player_type == 'PWF':
    draft_analyst_df['CHARACTERVADER'] *= forward_skill_df.loc["Power","Character"]
    draft_analyst_df['SKATINGVADER'] *= forward_skill_df.loc["Power","Skating"]
    draft_analyst_df['PUCKSKILLVADER'] *= forward_skill_df.loc["Power","Puck Skill"]
    draft_analyst_df['SHOOTINGVADER'] *= forward_skill_df.loc["Power","Shot"]
  elif player_type == 'PLY':
    draft_analyst_df['CHARACTERVADER'] *= forward_skill_df.loc["Playmaker","Character"]
    draft_analyst_df['SKATINGVADER'] *= forward_skill_df.loc["Playmaker","Skating"]
    draft_analyst_df['PUCKSKILLVADER'] *= forward_skill_df.loc["Playmaker","Puck Skill"]
    draft_analyst_df['SHOOTINGVADER'] *= forward_skill_df.loc["Playmaker","Shot"]
elif position == 'D':
  if player_type == 'TWD':
    draft_analyst_df['CHARACTERVADER'] *= defense_skill_df.loc["Two-Way","Character"]
    draft_analyst_df['SKATINGVADER'] *= defense_skill_df.loc["Two-Way","Skating"]
    draft_analyst_df['PUCKSKILLVADER'] *= defense_skill_df.loc["Two-Way","Puck Skill"]
    draft_analyst_df['SHOOTINGVADER'] *= defense_skill_df.loc["Two-Way","Shot"]
  elif player_type == 'OFD':
    draft_analyst_df['CHARACTERVADER'] *= defense_skill_df.loc["Offensive","Character"]
    draft_analyst_df['SKATINGVADER'] *= defense_skill_df.loc["Offensive","Skating"]
    draft_analyst_df['PUCKSKILLVADER'] *= defense_skill_df.loc["Offensive","Puck Skill"]
    draft_analyst_df['SHOOTINGVADER'] *= defense_skill_df.loc["Offensive","Shot"]
  elif player_type == 'DFD':
    draft_analyst_df['CHARACTERVADER'] *= defense_skill_df.loc["Defensive","Character"]
    draft_analyst_df['SKATINGVADER'] *= defense_skill_df.loc["Defensive","Skating"]
    draft_analyst_df['PUCKSKILLVADER'] *= defense_skill_df.loc["Defensive","Puck Skill"]
    draft_analyst_df['SHOOTINGVADER'] *= defense_skill_df.loc["Defensive","Shot"]

Please enter 'F' for forward or 'D' for defense: F
Please enter one of the following forward player types - 'TWF' for two-way forward, 'GRN' for energy player, 'PWF' for power forward, 'SNP' for sniper, 'PLY' for playmaker: PWF


In [ ]:
draft_analyst_df['AVERAGEVADER'] = draft_analyst_df[['CHARACTERVADER', 'SKATINGVADER', 'PUCKSKILLVADER', 'SHOOTINGVADER']].mean(axis=1)
draft_analyst_df.head()

# If you want to sort by average scores
#draft_analyst_df.sort_values(by=['AVERAGEVADER'], ascending=False)

,Unnamed: 0,RANK,NAME,SOURCE,CHARACTER,SKATING,PUCK SKILL,SHOOTING,CHARACTERVADER,SKATINGVADER,PUCKSKILLVADER,SHOOTINGVADER,AVERAGEVADER
0,0,1,"Lafreniere, Alexis",DraftAnalyst 2020.pdf,['Lafreniere also is a team-first leader who h...,['Lafreniere is an excellent skater in all dir...,"['What makes Lafreniere unique, however, is hi...",['From his deadly wrist shot to his superior p...,1.9280,0.9050,1.7746,0.7636,1.342800
1,1,2,"Byfield, Quinton",DraftAnalyst 2020.pdf,['Byfield combines obvious physical strengths ...,['He is as much a danger in tight spaces as he...,"['On the power play, Byfield threads the needl...","['Byfield, however, is exceptionally crafty wi...",1.9682,0.7506,1.9756,0.4764,1.292700
2,2,3,"Stutzle, Tim",DraftAnalyst 2020.pdf,['His quickness on or off the puck is evident ...,['His first step is explosive; his skating sty...,['His quickness on or off the puck is evident ...,['His quickness on or off the puck is evident ...,1.9622,0.7681,1.9194,1.9356,1.646325
3,3,4,"Perfetti, Cole",DraftAnalyst 2020.pdf,['A lethal scorer who possesses an arsenal of ...,['He is a strong skater with excellent balance...,"['Off the puck, he has very quick hands and a ...",['A lethal scorer who possesses an arsenal of ...,-0.0524,0.9578,0.9804,1.0846,0.742600
4,4,5,"Rossi, Marco",DraftAnalyst 2020.pdf,"['Rossi is a consistent breakaway threat, but ...","['Rossi is a consistent breakaway threat, but ...",['A dynamic playmaker with elite finishing ski...,['One of the more exciting forward prospects i...,1.9616,0.6310,1.5628,1.6864,1.460450


In [ ]:
draft_analyst_df.to_csv('final_score.csv')

In [ ]:
test_df = draft_analyst_df[draft_analyst_df.SOURCE == "DraftAnalyst 2020.pdf" ]
test_df.head()
test_df.iloc[1,11]

0.4764

Quantitative Combination Example with pick224.com statistics

In [ ]:
example_df = pd.DataFrame({
    'First Name': ['Quinton', 'Cole', 'Marco'],
    'Last Name': ['Byfield', 'Perfetti', 'Rossi'],
    'Games': [45, 61, 57],
    'Goals': [32, 37, 39],
    'SHOOTINGVADER': [test_df.iloc[1,11], test_df.iloc[3,11], test_df.iloc[4,11]]})
example_df.head()

While this is a very simple example, we could potentially interpret this information in a number of ways. For one, it could be that this scout was overly harsh on Byfield's shooting ability considering he had put up similar goal totals as Rossi and Perfetti in fewer games. Another way could be that the scout does not think Byfield's skills match his production, and perhaps also believes that his goal scoring from the OHL will not translate to the NHL, whereas the others have skills which are much better and could potentially translate.

The point of this example is that even in a simplistic model, we can directly combine both the VADER scores and traditional statistics (Games, Goals, etc.) to create insights that may have otherwise not been as easily spotted.